In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import min as spark_min, max as spark_max
from pyspark.sql.types import ArrayType, DoubleType

from delta.tables import DeltaTable

from sedona.spark import *

import random
import geopandas as gpd

In [0]:
# config
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
log_table = f"{catalog_dev}.{schema_dev}.logs_processed_gdbs"
bronze_table = f"{catalog_dev}.{schema_dev}.polygons_bronze"
silver_table = f"{catalog_dev}.{schema_dev}.polygons_silver"
buffer = 20

In [0]:
df_bronze = spark.read.table(bronze_table)
df_bronze.display(5)

In [0]:
def read_table_to_wkt() -> DataFrame:
    """
    Read a Spark DataFrame column containing WKT and return a Spark DataFrame with the
    """
    df_bronze = spark.read.table(bronze_table).withColumn("geometry", F.expr("ST_GeomFromWKT(geometry)"))
    return df_bronze

In [0]:
def make_envelope(df: DataFrame) -> DataFrame:
    """
    Make a envelope based on the geom.
    """
    return df.withColumn("envelope", F.expr("ST_Boundary(geometry)"))

In [0]:
def random_adjusted_bbox(envelope: list, output_size: int=512) -> list:
    """
    Generate a random bbox based on the envelope.
    """
    xmin, ymin, xmax, ymax = envelope
    poly_width = xmax - xmin
    poly_height = ymax - ymin

    if poly_width > output_size or poly_height > output_size:
        return None

    max_dx = output_size - poly_width
    max_dy = output_size - poly_height

    dx = random.uniform(0, max_dx)
    dy = random.uniform(0, max_dy)

    adjusted_xmin = xmin - dx
    adjusted_ymin = ymin - dy
    adjusted_xmax = adjusted_xmin + output_size
    adjusted_ymax = adjusted_ymin + output_size

    return [adjusted_xmin, adjusted_ymin, adjusted_xmax, adjusted_ymax]

In [0]:
def make_bbox(df: DataFrame, column_name: str) -> DataFrame: 
    """
    Make a bounding box from a Spark DataFrame based on the envelope.
    """
    df = df.withColumn(
    "bbox",
    F.expr(f"""
    array(
        ST_X(ST_Centroid(envelope)) - (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_Y(ST_Centroid(envelope)) - (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_X(ST_Centroid(envelope)) + (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_Y(ST_Centroid(envelope)) + (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer})
    )
    """) # Lager BBOX kolonne som kan brukes videre i WMSene på en enkel måte.
    )
    df = df.withColumn(
        "Polygons",
        F.expr("ST_MakeEnvelope(bbox[0], bbox[1], bbox[2], bbox[3])")
    ) # Lager polygons basert på bboxen 
    df = df.withColumn(
        "Adjusted_bbox",
        adjusted_bbox_udf(F.col("bbox"))
    ).drop("envelope")  # Dropper envelope kolonnen. Denne kolonnen brukes egt bare for å plotte bboxene. Strengt tatt ikke nødvendig å bruke.
    return df

In [0]:
def write_delta_table(sdf: DataFrame):
    """
    Write delta table from spark dataframe.
    """
    if not spark.catalog.tableExists(silver_table):
        sdf.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable(silver_table)
    else:
        delta_tbl = DeltaTable.forName(spark, silver_table)
        delta_tbl.alias("target").merge(
                    source=sdf.alias("source"),
                    condition="target.row_hash = source.row_hash"
                ).whenMatchedUpdate(
                    condition="target.row_hash != source.row_hash",
                    set={col: f"source.{col}" for col in sdf.columns}
                ).whenNotMatchedInsert(
                    values={col: f"source.{col}" for col in sdf.columns}
                ).execute()


In [0]:
def plot(df: DataFrame, column_name: str, plot_column: str):
    """
    Plot the bbox.
    """
    bbox_gdf = gpd.GeoDataFrame(
    df.toPandas(),
    geometry=column_name,
    crs="EPSG:25833",  # Bruker ETRS89 / UTM sone 33N (Norge)
    )
    return bbox_gdf

In [0]:
adjusted_bbox_udf = F.udf(lambda envelope: random_adjusted_bbox(envelope), ArrayType(DoubleType()))

df = read_table_to_wkt()
df = make_envelope(df)
df = make_bbox(df, "envelope")
gdf = plot(df, "Polygons", "Vegtyper")
write_delta_table(df)


In [0]:
df.display()

In [0]:
# Uncomment under hvis man ønsker å se på plotten
# gdf.explore(column="Vegtyper", tooltip="Vegtyper", popup=True, cmap="Set1")